In [ ]:
from py123d.datatypes.scene.arrow.arrow_scene_builder import ArrowSceneBuilder
from py123d.datatypes.scene.scene_filter import SceneFilter

from py123d.common.multithreading.worker_sequential import Sequential
from py123d.datatypes.sensors.pinhole_camera import PinholeCameraType

KITTI360_DATA_ROOT = "/home/daniel/kitti_360/KITTI-360"

In [ ]:
# splits = ["kitti360"]
# splits = ["nuscenes-mini_val", "nuscenes-mini_train"]
# splits = ["nuplan-mini_test", "nuplan-mini_train", "nuplan-mini_val"]
# splits = ["nuplan_private_test"]
# splits = ["carla_test"]
splits = ["wopd_val"]
# splits = ["av2-sensor_train"]
# splits = ["pandaset_test", "pandaset_val", "pandaset_train"]
# log_names = ["2021.08.24.13.12.55_veh-45_00386_00472"]
# log_names = ["2013_05_28_drive_0000_sync"]
# log_names = ["2013_05_28_drive_0000_sync"]
log_names = None
scene_uuids = ["9727e2b3-46b0-51bd-84a9-c516c0993045"]

scene_filter = SceneFilter(
    split_names=splits,
    log_names=log_names,
    scene_uuids=scene_uuids,
    duration_s=None,
    history_s=0.0,
    timestamp_threshold_s=None,
    shuffle=True,
    # camera_types=[PinholeCameraType.CAM_F0],
)
scene_builder = ArrowSceneBuilder()
worker = Sequential()
scenes = scene_builder.get_scenes(scene_filter, worker)

scenes  = [scene for scene in scenes if scene.uuid in scene_uuids]
print(f"Found {len(scenes)} scenes")


In [ ]:
from matplotlib import pyplot as plt
from py123d.datatypes.scene.abstract_scene import AbstractScene
from py123d.visualization.matplotlib.camera import add_box_detections_to_camera_ax, add_camera_ax
import imageio
import numpy as np

iteration = 0
scene = scenes[0]

scene: AbstractScene
fps = 15  # frames per second
output_file = f"camera_{scene.log_metadata.split}_{scene.uuid}.mp4"

writer = imageio.get_writer(output_file, fps=fps)

scale = 3.0
fig, ax = plt.subplots(2, 3, figsize=(scale * 6, scale * 2.5))


camera_type = PinholeCameraType.CAM_F0

for i in range(scene.number_of_iterations):
    camera = scene.get_camera_at_iteration(i, camera_type)
    box_detections = scene.get_box_detections_at_iteration(i)
    ego_state = scene.get_ego_state_at_iteration(i)

    _, image = add_box_detections_to_camera_ax(
        None,
        camera,
        box_detections,
        ego_state,
        return_image=True,
    )
    writer.append_data(image)

writer.close()